### Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xgboost
!pip install lightgbm

  Using cached lightgbm-4.6.0-py3-none-win_amd64.whl.metadata (17 kB)
Using cached lightgbm-4.6.0-py3-none-win_amd64.whl (1.5 MB)


In [3]:
import pandas as pd
from sklearn import linear_model, metrics, preprocessing, tree, model_selection, feature_selection, ensemble
import numpy as np
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter
import lightgbm as lgb

import nltk
import string
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
try:
    stopwords.words('english')
except LookupError:
    nltk.download('stopwords')
try:
    WordNetLemmatizer().lemmatize('cats')
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')
try:
    SentimentIntensityAnalyzer().polarity_scores("hello")
except LookupError:
    nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ivanj\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ivanj\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ivanj\AppData\Roaming\nltk_data...
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ivanj\AppData\Roaming\nltk_data...


### Loading training and tesing data

In [5]:
# train_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/train.csv')
# test_set = pd.read_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/test.csv')

train_set = pd.read_csv('train.csv')
test_set = pd.read_csv('test.csv')

### Data Processing & EDA

In [6]:
train_set.head()
orgin_instrances = train_set.shape[0]

In [7]:
train_set.describe()

,favourites_count,followers_count,friends_count,id,statuses_count,average_tweets_per_day,account_age_days,target
count,26206.000000,2.620600e+04,2.620600e+04,2.620600e+04,2.620600e+04,26206.000000,26206.000000,26206.000000
mean,12159.971342,3.732983e+05,4.427275e+03,1.253306e+17,2.072485e+04,6.754636,3017.870831,0.335152
std,33628.948536,2.515996e+06,4.602204e+04,3.035728e+17,6.345569e+04,22.782745,1018.958572,0.472053
min,0.000000,0.000000e+00,0.000000e+00,5.860000e+02,0.000000e+00,0.000000,483.000000,0.000000
25%,347.000000,3.400000e+01,3.400000e+01,9.799366e+07,1.331000e+03,0.486250,2302.000000,0.000000
50%,2037.000000,3.540000e+02,2.920000e+02,3.997210e+08,4.143500e+03,1.464000,3218.000000,0.000000
75%,8751.750000,8.241500e+03,8.780000e+02,2.480241e+09,1.679150e+04,5.460000,3895.000000,1.000000
max,839929.000000,1.216415e+08,1.641962e+06,1.120974e+18,2.771910e+06,1191.286000,5151.000000,1.000000


In [8]:
train_set.describe()

,favourites_count,followers_count,friends_count,id,statuses_count,average_tweets_per_day,account_age_days,target
count,26206.000000,2.620600e+04,2.620600e+04,2.620600e+04,2.620600e+04,26206.000000,26206.000000,26206.000000
mean,12159.971342,3.732983e+05,4.427275e+03,1.253306e+17,2.072485e+04,6.754636,3017.870831,0.335152
std,33628.948536,2.515996e+06,4.602204e+04,3.035728e+17,6.345569e+04,22.782745,1018.958572,0.472053
min,0.000000,0.000000e+00,0.000000e+00,5.860000e+02,0.000000e+00,0.000000,483.000000,0.000000
25%,347.000000,3.400000e+01,3.400000e+01,9.799366e+07,1.331000e+03,0.486250,2302.000000,0.000000
50%,2037.000000,3.540000e+02,2.920000e+02,3.997210e+08,4.143500e+03,1.464000,3218.000000,0.000000
75%,8751.750000,8.241500e+03,8.780000e+02,2.480241e+09,1.679150e+04,5.460000,3895.000000,1.000000
max,839929.000000,1.216415e+08,1.641962e+06,1.120974e+18,2.771910e+06,1191.286000,5151.000000,1.000000


In [9]:
test_set.describe()

,index,favourites_count,followers_count,friends_count,id,statuses_count,average_tweets_per_day,account_age_days
count,11232.000000,11232.000000,1.123200e+04,1.123200e+04,1.123200e+04,1.123200e+04,11232.000000,11232.000000
mean,5615.500000,12633.582176,3.633372e+05,4.489438e+03,1.147413e+17,2.188658e+04,7.211383,3032.060808
std,3242.543446,34600.722428,2.362189e+06,5.692586e+04,2.928501e+17,7.559739e+04,28.332281,1001.284291
min,0.000000,0.000000,0.000000e+00,0.000000e+00,4.180000e+02,0.000000e+00,0.000000,500.000000
25%,2807.750000,397.000000,3.700000e+01,4.375000e+01,1.019660e+08,1.422750e+03,0.511750,2345.000000
50%,5615.500000,2132.000000,3.900000e+02,3.060000e+02,3.969268e+08,4.392500e+03,1.553000,3222.000000
75%,8423.250000,9221.250000,8.979500e+03,9.232500e+02,2.403527e+09,1.800525e+04,5.676750,3879.000000
max,11231.000000,885123.000000,1.084423e+08,4.343060e+06,1.114681e+18,2.497571e+06,1269.055000,5158.000000


In [11]:
print("Size of Training Instances:", train_set.shape[0])
print("Sice of Features:", train_set.shape[1])
print(train_set.info())

Size of Training Instances: 26206
Sice of Features: 19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26206 entries, 0 to 26205
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   created_at                    26206 non-null  object 
 1   default_profile               26206 non-null  bool   
 2   default_profile_image         26206 non-null  bool   
 3   description                   21115 non-null  object 
 4   favourites_count              26206 non-null  int64  
 5   followers_count               26206 non-null  int64  
 6   friends_count                 26206 non-null  int64  
 7   geo_enabled                   26206 non-null  bool   
 8   id                            26206 non-null  int64  
 9   lang                          20618 non-null  object 
 10  location                      26204 non-null  object 
 11  profile_background_image_url  22971 non-null  object 
 12  profi

In [12]:
print("Size of Testing Instances:", test_set.shape[0])
print("Sice of Features:", test_set.shape[1])
print(test_set.info())

Size of Testing Instances: 11232
Sice of Features: 19
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11232 entries, 0 to 11231
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   index                         11232 non-null  int64  
 1   created_at                    11232 non-null  object 
 2   default_profile               11232 non-null  bool   
 3   default_profile_image         11232 non-null  bool   
 4   description                   9066 non-null   object 
 5   favourites_count              11232 non-null  int64  
 6   followers_count               11232 non-null  int64  
 7   friends_count                 11232 non-null  int64  
 8   geo_enabled                   11232 non-null  bool   
 9   id                            11232 non-null  int64  
 10  lang                          8863 non-null   object 
 11  location                      11230 non-null  object 
 12  profil

In [13]:
train_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)
test_set.drop(columns=['created_at','id', 'profile_background_image_url', 'profile_image_url', 'screen_name', 'location'], inplace=True)

In [17]:
train_set['lang'] = train_set['lang'].fillna('unknown')
test_set['lang'] = test_set['lang'].fillna('unknown')
train_set['lang'] = train_set['lang'].astype('category')
test_set['lang'] = test_set['lang'].astype('category')

In [15]:
train_set['default_profile'] = train_set['default_profile'].astype(int)
train_set['default_profile_image'] = train_set['default_profile_image'].astype(int)
train_set['geo_enabled'] = train_set['geo_enabled'].astype(int)
train_set['verified'] = train_set['verified'].astype(int)

test_set['default_profile'] = test_set['default_profile'].astype(int)
test_set['default_profile_image'] = test_set['default_profile_image'].astype(int)
test_set['geo_enabled'] = test_set['geo_enabled'].astype(int)
test_set['verified'] = test_set['verified'].astype(int)

In [16]:
train_set['follower_ratio'] = train_set['followers_count'] / (train_set['friends_count']+1)
test_set['follower_ratio'] = test_set['followers_count'] / (test_set['friends_count']+1)
train_set['favourite_per_tweet'] = train_set['favourites_count'] / (train_set['statuses_count']+1)
test_set['favourite_per_tweet'] = test_set['favourites_count'] / (test_set['statuses_count']+1)
train_set['favourite_per_day'] = train_set['favourites_count'] / (train_set['average_tweets_per_day']+1)
test_set['favourite_per_day'] = test_set['favourites_count'] / (test_set['average_tweets_per_day']+1)

In [92]:
# train_set.drop(columns=['statuses_count'], inplace=True)
# test_set.drop(columns=['statuses_count'], inplace=True)

### NLP

In [18]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    if pd.isna(text) or text == "":
        return ""
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

train_set['description'] = train_set['description'].fillna('')
train_set['description'] = train_set['description'].apply(preprocess_text)
test_set['description'] = test_set['description'].fillna('')
test_set['description'] = test_set['description'].apply(preprocess_text)

In [20]:
train_set['desc_len'] = train_set['description'].apply(len)
train_set['desc_wordcount'] = train_set['description'].apply(lambda x: len(x.split()))
test_set['desc_len'] = test_set['description'].apply(len)
test_set['desc_wordcount'] = test_set['description'].apply(lambda x: len(x.split()))

In [21]:
sid = SentimentIntensityAnalyzer()
def get_sentiment_scores(text):
    if pd.isna(text) or text == "":
        return 0.0
    return sid.polarity_scores(text)['compound']

train_set['desc_sentiment'] = train_set['description'].apply(get_sentiment_scores)
test_set['desc_sentiment'] = test_set['description'].apply(get_sentiment_scores)

#### Data Split

In [22]:
train_set = train_set.drop(columns=['description'])
test_set = test_set.drop(columns=['description'])

In [23]:
num_majority = train_set[train_set['target'] == 0].shape[0]
num_minority = train_set[train_set['target'] == 1].shape[0]

In [24]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(train_set.drop(columns=['target']), train_set['target'], test_size=0.2, shuffle = True, random_state=42, stratify=train_set['target'])

In [25]:
counter = Counter(y_train)
estimate_scale_pos_weight = counter[0] / counter[1]

In [26]:
train_predictors = train_set.drop(columns=['target'])
train_target = train_set['target']

In [27]:
counter_train = Counter(train_target)
estimate_scale_pos_weight_train = counter_train[0] / counter_train[1]

In [28]:
x_train

,default_profile,default_profile_image,favourites_count,followers_count,friends_count,geo_enabled,lang,statuses_count,verified,average_tweets_per_day,account_age_days,follower_ratio,favourite_per_tweet,favourite_per_day,desc_len,desc_wordcount,desc_sentiment
16919,0,0,1534,45,222,0,pt,2759,0,0.705,3915,0.201794,0.555797,899.706745,80,18,0.0000
3662,0,0,2207,11557,820,1,en,7577,0,1.984,3820,14.076736,0.291238,739.611260,33,6,0.0000
6669,0,0,738,36684,185,1,ar,141155,0,42.414,3328,197.225806,0.005228,16.999125,46,6,0.0000
7412,0,0,24553,970,968,1,en,10305,0,2.783,3703,1.001032,2.382399,6490.351573,27,3,0.0000
8506,0,0,8942,1898,807,1,en,10848,1,2.601,4170,2.349010,0.824223,2483.199111,64,9,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,0,0,2768,295,295,1,en,13869,0,4.086,3394,0.996622,0.199567,544.239088,13,3,0.0762
92,0,0,63616,883,742,1,it,52246,0,25.954,2013,1.188425,1.217601,2360.169177,20,4,0.0000
8298,0,0,3256,341,186,1,en,1580,0,0.386,4092,1.823529,2.059456,2349.206349,66,12,0.4215
14834,1,0,1332,8,31,0,en,1333,0,0.912,1462,0.250000,0.998501,696.652720,21,2,0.0000


### LGB

In [50]:
lgbm_classifier = lgb.LGBMClassifier(
    objective='binary',
    metric='auc',
    n_estimators=500,
    learning_rate=0.1,
    num_leaves=31,
    max_depth=-1,
    min_child_samples=20,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1,
    device='cuda',
    gpu_platform_id=0,
    gpu_device_id=0,
    categorical_feature=['lang']
)

In [51]:
cv_strategy = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [52]:
param_grid_lgbm = {
    'n_estimators': [400, 500, 600],
    'learning_rate': [0.001, 0.01, 0.02],
    'num_leaves': [15, 30, 45]

}

In [53]:
cv_strategy = model_selection.StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [54]:
grid_search_lgb = model_selection.GridSearchCV(
    estimator=lgbm_classifier,
    param_grid=param_grid_lgbm,
    scoring='roc_auc',
    cv=cv_strategy,
    n_jobs=1,
    verbose=2
)

In [55]:
grid_search_lgb.fit(x_train,y_train)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature

c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in para

[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12545
[CV] END learning_rate=0.001, n_estimators=400, num_leaves=30; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12545
[CV] END learning_rate=0.001, n_estimators=400, num_leaves=30; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6324, number of negative: 12544
[CV] END learning_rate=0.001, n_estimators=400, num_leaves=30; total

c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in para

[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[CV] END learning_rate=0.001, n_estimators=500, num_leaves=15; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[CV] END learning_rate=0.001, n_estimators=500, num_leaves=15; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of neg

...

[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12545
[CV] END learning_rate=0.02, n_estimators=500, num_leaves=45; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12545
[CV] END learning_rate=0.02, n_estimators=500, num_leaves=45; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6324, number of negative: 12544
[CV] END learning_rate=0.02, n_estimators=500, num_leaves=45; total ti

c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in para

[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[CV] END learning_rate=0.02, n_estimators=600, num_leaves=30; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negative: 12544
[CV] END learning_rate=0.02, n_estimators=600, num_leaves=30; total time=   0.0s
[LightGBM] [Warning] Using sparse features with CUDA is currently not supported.
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=6 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 6323, number of negat

c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2137: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\basic.py:2159: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in para

ValueError: 
All the 270 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
270 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\sklearn.py", line 1560, in fit
    super().fit(
    ~~~~~~~~~~~^
        X,
        ^^
    ...<12 lines>...
        init_model=init_model,
        ^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "c:\Users\ivanj\AppData\Local\Programs\Python\Python313\Lib\site-packages\lightgbm\sklearn.py", line 1049, in fit
  

In [110]:
best_lgb_model = grid_search_lgb.best_estimator_
y_pred_train_proba_lgb= best_lgb_model.predict_proba(x_train)[:, 1]
train_auc_lgb = metrics.roc_auc_score(y_train, y_pred_train_proba_lgb)
print(f"Train AUC for tuned XGBoost: {train_auc_lgb:.4f}")
y_pred_test_proba_lgb = best_lgb_model.predict_proba(x_test)[:, 1]
test_auc_lgb = metrics.roc_auc_score(y_test, y_pred_test_proba_lgb)
print(f"Test AUC for tuned XGBoost: {test_auc_lgb:.4f}")

Train AUC for tuned XGBoost: 0.9838
Test AUC for tuned XGBoost: 0.9483


In [111]:
importances = best_lgb_model.feature_importances_
feature_names = x_train.columns
feature_importance = pd.DataFrame({'feature': feature_names, 'importance': importances})
feature_importance = feature_importance.sort_values('importance', ascending=False)
print(feature_importance)

                   feature  importance
8         account_age_days        3452
7   average_tweets_per_day        3137
10     favourite_per_tweet        2909
2          followers_count        2652
3            friends_count        2510
1         favourites_count        2317
11       favourite_per_day        2183
9           follower_ratio        1965
12                desc_len        1604
14          desc_sentiment        1158
13          desc_wordcount         921
5                     lang         774
4              geo_enabled         324
0          default_profile         252
6                 verified         242


In [ ]:
x_train = x_train.drop(columns=['default_profile_image'])
x_test = x_test.drop(columns=['default_profile_image'])

In [115]:
train_predictors = train_predictors.drop(columns=['default_profile_image'])

In [49]:
# x_train = x_train.drop(columns=['geo_enabled', 'default_profile', 'verified'])
# x_test = x_test.drop(columns=['geo_enabled', 'default_profile', 'verified'])

In [116]:
grid_search_lgb.fit(train_predictors, train_target)

Fitting 10 folds for each of 27 candidates, totalling 270 fits
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=5 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2510
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 15
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.036296 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   3.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=5 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2519
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 15
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.022401 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.001, n_estimators=400, num_leaves=15; total time=   1.1s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=5 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7904, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2517
[LightGBM] [Info] Number of data points in the train set: 23585, number of used features: 15
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.001266 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335128 -> initscore=-0.685081
[LightGBM] [Info] Start training from score -0.685081


...

[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.6s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=5 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 7905, number of negative: 15681
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2514
[LightGBM] [Info] Number of data points in the train set: 23586, number of used features: 15
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.36 MB) transferred to GPU in 0.001206 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335156 -> initscore=-0.684954
[LightGBM] [Info] Start training from score -0.684954


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


[CV] END learning_rate=0.02, n_estimators=600, num_leaves=45; total time=   3.2s
[LightGBM] [Warning] categorical_feature is set=lang, categorical_column=5 will be ignored. Current value: categorical_feature=lang
[LightGBM] [Info] Number of positive: 8783, number of negative: 17423
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 2525
[LightGBM] [Info] Number of data points in the train set: 26206, number of used features: 15
[LightGBM] [Info] Using GPU Device: Tesla T4, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 13 dense feature groups (0.40 MB) transferred to GPU in 0.001239 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.335152 -> initscore=-0.684973
[LightGBM] [Info] Start training from score -0.684973


/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2118: UserWarning: categorical_feature keyword has been found in `params` and will be ignored.
Please use categorical_feature argument of the Dataset constructor to pass this parameter.
  _log_warning(
/usr/local/lib/python3.11/dist-packages/lightgbm/basic.py:2140: UserWarning: categorical_feature in param dict is overridden.
  _log_warning(f"{cat_alias} in param dict is overridden.")


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=LGBMClassifier(categorical_feature=['lang'],
                                      colsample_bytree=0.8, device='gpu',
                                      metric='auc', n_estimators=500, n_jobs=-1,
                                      objective='binary', random_state=42,
                                      subsample=0.8),
             n_jobs=1,
             param_grid={'learning_rate': [0.001, 0.01, 0.02],
                         'n_estimators': [400, 500, 600],
                         'num_leaves': [15, 30, 45]},
             scoring='roc_auc', verbose=2)

In [117]:
best_lgb_model = grid_search_lgb.best_estimator_
train_target_proba = best_lgb_model.predict_proba(train_predictors)[:, 1]
train_auc_lgb = metrics.roc_auc_score(train_target, train_target_proba)
print(f"Train AUC for tuned XGBoost: {train_auc_lgb:.4f}")

Train AUC for tuned XGBoost: 0.9797


In [119]:
test_set = test_set.drop(columns=['default_profile_image'])

In [120]:
test_set_predictors = test_set.drop(columns=['index'])
test_set_target_proba = best_lgb_model.predict_proba(test_set_predictors)[:, 1]

test_set_prob = pd.DataFrame({'index': test_set['index'], 'target': test_set_target_proba})
test_set_prob.to_csv('/content/drive/MyDrive/10 Github/CS610_AML/01 Assignment 1/lgb_test_set_prob_7.csv', index=False)